In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os
import json

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

molecule_selector = widgets.MoleculeSelectionWidget()
molecule_selector.load_dropdown_box()

chemist_selector = widgets.ChemistSelectionWidget()
chemist_selector.load_dropdown_box()

storage_selector = widgets.StorageSelectionWidget()
storage_selector.load_dropdown_box()

supplier_selector = widgets.SupplierSelectionWidget()
supplier_selector.load_dropdown_box()

substance_widgets = widgets.ObjectPropertiesWidgets("Substance")

substance_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def create_substance_openbis(b):
    if molecule_selector.dropdown_boxes.children[0].value == -1:
        display(utils.Javascript(data = "alert('Select a molecule.')"))
        return
    
    # Check if the molecule is already in openBIS
    molecules_empa_id_openbis = [f"{substance.props['empa_number']}-{substance.props['batch']}" for substance in utils.get_openbis_objects(OPENBIS_SESSION, type ="SUBSTANCE")]
        
    object_properties = {}
    for prop in CONFIG["objects"]["Substance"]["properties"]:
        if prop == "evaporator_slot":
            object_properties[prop] = json.dumps({"evaporator_number": substance_widgets.properties_widgets[prop].children[0].value, 
                                                    "details": substance_widgets.properties_widgets[prop].children[1].value})
        elif CONFIG["properties"][prop]["property_type"] == "QUANTITY_VALUE":
            object_properties[prop] = json.dumps({"has_value": substance_widgets.properties_widgets[prop].children[0].value, 
                                                    "has_unit": substance_widgets.properties_widgets[prop].children[1].value})
        elif CONFIG["properties"][prop]["property_type"] == "DATE":
            object_properties[prop] = str(substance_widgets.properties_widgets[prop].value)
        else:
            object_properties[prop] = substance_widgets.properties_widgets[prop].value
    
    molecule_empa_id = f"{object_properties['empa_number']}-{object_properties['batch']}"
    if molecule_empa_id in molecules_empa_id_openbis:
        display(utils.Javascript(data = "alert('Substance is already in openBIS!')"))
        return
    
    object_properties["has_molecule"] = molecule_selector.dropdown_boxes.children[0].value
    object_properties["has_chemist"] = chemist_selector.dropdown_boxes.children[0].value
    object_properties["has_supplier"] = supplier_selector.dropdown_boxes.children[0].value
    object_properties["has_storage"] = storage_selector.dropdown_boxes.children[0].value

    results_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="SUBSTANCE", 
        collection="/MATERIALS/SUBSTANCES/SUBSTANCE_COLLECTION", 
        props = object_properties
    )
    upload_datasets(results_object, substance_support_files)
    
    print("Upload successful!")

def upload_datasets(method_object, support_files_widget):
    for filename in support_files_widget.value:
        file_info = support_files_widget.value[filename]
        utils.save_file(file_info['content'], filename)
        OPENBIS_SESSION.new_dataset(type = 'RAW_DATA', sample = method_object, files = [filename]).save()
        os.remove(filename)

def checkbox_on_change(checkbox, textbox):
    def enable_textbox(change):
        if checkbox.value:
            textbox.disabled = False
        else:
            textbox.disabled = True
    checkbox.observe(enable_textbox, names = "value")

# Create substance

## Select molecule

In [ ]:
molecule_selector.dropdown_boxes.children[0].observe(molecule_selector.load_molecule_metadata, names = "value")
display(molecule_selector)

## Select chemist

In [ ]:
display(chemist_selector)

## Select storage

In [ ]:
display(storage_selector)

## Select supplier

In [ ]:
display(supplier_selector)

## Properties

In [ ]:
checkbox_on_change(substance_widgets.properties_widgets["hazardous"], substance_widgets.properties_widgets["hazardous_specification"])
checkbox_on_change(substance_widgets.properties_widgets["other_storage_condition"], substance_widgets.properties_widgets["other_storage_condition_specification"])
display(substance_widgets)

## Support files

In [ ]:
display(substance_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_substance_openbis)
quit_button.on_click(close_notebook)